In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import jieba
import pandas as pd
import re
import numpy as np
import sys

## Define tokenizer and stop words.

def tokenize(text):
    words = jieba.lcut(text)
    return words

stopWords = ["。", "，", "？"]

## Split customer questions

f = open('chatbot.txt', 'r')
fileContent = f.read()
contentSplit = fileContent.splitlines()
clientQuestions = [x.split("):  ")[1] for x in contentSplit if ("佩爱旗舰店" not in x) and ("):" in x)]

for i in range(len(clientQuestions)):
    if "[" in clientQuestions[i] and "]" in clientQuestions[i]:
        clientQuestions[i] = clientQuestions[i].split("[")[0] + clientQuestions[i].split("[")[1].split("]")[1]
        
clientQuestions = [x for x in clientQuestions if ("http" not in x) and len(x) != 0]

## TF-IDF process

vectorizer = TfidfVectorizer(tokenizer = tokenize,
                            stop_words = stopWords)
matrix = vectorizer.fit_transform(clientQuestions)

del fileContent, contentSplit

## Features

originalFeatures = vectorizer.get_feature_names()
columnIndex = []
columnNames = []

for i in range(len(originalFeatures)):
    if len(re.findall(r'[\u4e00-\u9fff]+', originalFeatures[i])) != 0:
        columnIndex.append(i)
        columnNames.append(originalFeatures[i])

## Extract features        

wordMatrix = np.array(matrix.toarray()).T[columnIndex]

del matrix

## Clusterring 

clusters = 50
kmeans = np.array(KMeans(n_clusters=clusters).fit(wordMatrix.T[:10000]).labels_)

## Get classes

classes = []
for i in range(clusters):
    ind = np.where(kmeans == i)[0]
    names = []
    for j in ind:
        names.append(clientQuestions[j])
    classes.append(names)
    
for i in range(clusters):
    f = open("Class_%d.txt" %(i),'w')
    newClass = map(lambda x:x+'\n', classes[i])
    f.writelines(newClass)
    f.close()

# features = [x for x in vectorizer.get_feature_names() if len(re.findall(r'[\u4e00-\u9fff]+', x)) != 0]
# wordDf = pd.DataFrame(matrix.toarray(),
#                       columns = vectorizer.get_feature_names())
# wordVector = wordDf[features]